In [1]:
from AOC_utils import get_day
import numpy as np

day = 21
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Day 21 input already downloaded


['...................................................................................................................................',
 '...............##....#.......#.....#..#...............#....#..........#........#.........#....#.............#.##.#.......#.#.......',
 '..............#..........#....##.#..........#......###.#.##...............#....#....#.#........##.....#..#.##..............#...##..',
 '........##....#..#.......#...#.#.........#.#.......#......#..............#.........#.........#....#.....#..#....##..##......#......',
 '......#..#...#...##.........#....##...#.#......#..........................#.....#...#..............................#..........#....',
 '..#.#..#...............#...........##.....#.#.#.................#...........#.....##.#..#...............#....#.#...#.........#...#.',
 '....##...#.#............#.#.....#..........#.#.#................#................##.#...#.....#........#....#...#......#...#.......']

In [2]:
example = '''
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
'''.split('\n')[1:-1]

In [3]:
len(input_data)

131

In [4]:
import pandas as pd

In [5]:
def parse_data(data, steps=26501365):

    grid = np.array([list(line) for line in data])
    grid_size = grid.shape
    # duplicate grid to make a 3x3 grid of grids
    # expand_by = 15
    # grid = np.concatenate([grid for _ in range(expand_by)], axis=-1)
    # grid = np.concatenate([grid for _ in range(expand_by)], axis=0)
    
    # for line in grid:
    #     print(''.join(line))

    # find the central S
    # S = np.argwhere(grid == 'S')[(expand_by ** 2) // 2]
    S = np.argwhere(grid == 'S')[0]

    # replace S with garden plot
    grid[S[0], S[1]] = '.'
    rocks = np.argwhere(grid == '#')
    is_not_rock_grid = grid != '#'

    counts_at_steps = []
    cache = {}

    directions = np.array([[0,1], [1,0], [0,-1], [-1,0]]).T[None,:,:]

    current_steps = np.array([S,])
    dataset_length = 1000
    for i in range(dataset_length): # only need to go far enough to build a dataset
            current_steps = current_steps[:,:,None] + directions
            current_steps = current_steps.transpose(0,2,1).reshape(-1,2)
            modulo_current_steps = current_steps % grid_size
            current_steps = current_steps[is_not_rock_grid[modulo_current_steps[:,0], modulo_current_steps[:,1]]]
            current_steps = np.unique(current_steps, axis=0)
        
            if i % 100 == 0:
                print((i / dataset_length) * 100, "%")
            counts_at_steps.append(len(current_steps))

    data_range = np.arange(dataset_length) + 1
    counts_at_steps = np.array(counts_at_steps)

    p, res, rank, sv, rcond = np.polyfit(data_range[grid_size[0]*2:].astype(np.float64), 
                    counts_at_steps[grid_size[0]*2:].astype(np.float64), 2, full=True)
    def solve(x):
        y = (p[0] * (x**2)) + (p[1] * x) + p[2]
        return y
    
    _diff = counts_at_steps - solve(data_range)
    p2, res2, rank2, sv2, rcond = np.polyfit(data_range[(steps-1) % grid_size[0]::grid_size[0]][1:].astype(np.float64), 
                    _diff[(steps-1) % grid_size[0]::grid_size[0]][1:], 2, full=True)

    def solve2(x):
        y = (p2[0] * (x**2)) + (p2[1] * x) + p2[2]
        return y

    total2 = solve(steps) + solve2(steps)
    print("part 2:", total2)
    return data_range, counts_at_steps

In [6]:
data_range, counts_at_steps = parse_data(input_data, 26501365)

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
597724705324407.8
part 2: 598044246091826.1


In [7]:
import plotly
import plotly.graph_objs as go
# plot data_range vs counts_at_steps
trace = go.Scatter(
    x = data_range,
    y = counts_at_steps,
    mode = 'markers'
)
fig = go.Figure(data=[trace])
fig.update_layout(template='plotly_dark')
fig.show()

In [8]:
p, res, rank, sv, rcond = np.polyfit(np.array(data_range[len(input_data)*2:]).astype(np.float64), 
                    np.array(counts_at_steps[len(input_data)*2:]).astype(np.float64), 2, full=True)

In [9]:
def solve(x):
    y = p[2] + (p[1] * x) + (p[0] * (x**2))
    return y

In [10]:
solve_diff = counts_at_steps - solve(np.array(data_range))

In [11]:
n_steps = 26501365 - 1

In [12]:
import plotly
import plotly.graph_objs as go
# plot data_range vs counts_at_steps
trace = go.Scatter(
    x = data_range,
    y = counts_at_steps - solve(np.array(data_range)),
    mode = 'markers'
)
# plot a green dot at every n_steps % len(input_data)th point
trace2 = go.Scatter(
    x = data_range[n_steps % len(input_data)::len(input_data)],
    y = counts_at_steps[n_steps % len(input_data)::len(input_data)] - solve(np.array(data_range[n_steps % len(input_data)::len(input_data)])),
    mode = 'markers',
    marker=dict(
        color='green',
        size=10,
    )
)
fig = go.Figure(data=[trace, trace2])
fig.update_layout(template='plotly_dark')
fig.show()

In [13]:
p2, res2, rank2, sv2, rcond = np.polyfit(np.array(data_range)[n_steps % len(input_data)::len(input_data)][1:], 
                    solve_diff[n_steps % len(input_data)::len(input_data)][1:], 2, full=True)

In [14]:
def solve2(x):
    y = p2[2] + (p2[1] * x) + (p2[0] * (x**2))
    return y

In [15]:
import plotly
import plotly.graph_objs as go
# plot data_range vs counts_at_steps
trace = go.Scatter(
    x = data_range,
    y = counts_at_steps - (solve(np.array(data_range)) + solve2(np.array(data_range))),
    mode = 'markers'
)
# plot a green dot at every n_steps % len(input_data)th point
trace2 = go.Scatter(
    x = data_range[n_steps % len(input_data)::len(input_data)],
    y = (counts_at_steps - (solve(np.array(data_range)) + solve2(np.array(data_range))))[n_steps % len(input_data)::len(input_data)],
    mode = 'markers',
    marker=dict(
        color='green',
        size=10,
    )
)
fig = go.Figure(data=[trace, trace2])
fig.update_layout(template='plotly_dark')
fig.show()

In [16]:
solve(26501365) + solve2(26501365)

598044246091826.1